In [ ]:
import pickle
import numpy as np

# Load the pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textebd18.pkl', 'rb') as file:
    data = pickle.load(file)

# Find the length of the longest audio embedding vector
max_length = max(len(item['embeddings']) for item in data)

# Pad the embedding vectors to the length of the longest vector
for item in data:
    embeddings = item['embeddings']
    padding_length = max_length - len(embeddings)
    padded_embeddings = np.pad(embeddings, (0, padding_length), mode='constant')
    item['embeddings'] = padded_embeddings

# Save the processed data to a new pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
# Load the pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as file:
    data = pickle.load(file)

In [ ]:
videos = pd.DataFrame(data)

In [ ]:
for i in videos['embeddings']:
    print(i.shape)

(57200,)
(57200,)
(57200,)
(57200,)


In [ ]:
import pickle
import numpy as np

# Load the pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videoebd18.pickle', 'rb') as file:
    data = pickle.load(file)

# Find the length of the longest audio embedding vector
max_length = 495360

# Pad the embedding vectors to the length of the longest vector
for item in data:
    embeddings = item['embeddings']
    padding_length = max_length - len(embeddings)
    padded_embeddings = np.pad(embeddings, (0, padding_length), mode='constant')
    item['embeddings'] = padded_embeddings

# Save the processed data to a new pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'wb') as file:
    pickle.dump(data, file)

In [ ]:
import pickle

# Load the embeddings from the pickle file
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

import pandas as pd
textpkl = pd.DataFrame(text_data)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

videopkl = pd.DataFrame(video_data)

In [ ]:
text_data

[{'filename': 'assets-_1DDhUnyvwY.txt',
  'embeddings': array([-0.09502907, -0.1150557 ,  0.2358259 , ...,  0.25622135,
         -0.38683465, -0.13876839], dtype=float32)},
 {'filename': 'assets-_0F7wFPRBmY.txt',
  'embeddings': array([-0.27596277, -0.51922   , -0.19767228, ...,  0.        ,
          0.        ,  0.        ], dtype=float32)},
 {'filename': 'assets-_1cHSTAv6aw.txt',
  'embeddings': array([-0.48289317, -0.3653778 ,  0.19714946, ...,  0.        ,
          0.        ,  0.        ], dtype=float32)},
 {'filename': 'assets-__5k7e0f3r4.txt',
  'embeddings': array([-0.9525771 ,  0.19983402,  0.39927948, ...,  0.        ,
          0.        ,  0.        ], dtype=float32)}]

**ALIGNMENT**

In [ ]:
print(textpkl['embeddings'][0],";",videopkl['filename'][0])

[-0.09502907 -0.1150557   0.2358259  ...  0.25622135 -0.38683465
 -0.13876839] ; video-_0F7wFPRBmY.mp4


In [ ]:
print(textpkl['filename'][1],";",videopkl['embeddings'][1])

assets-_0F7wFPRBmY.txt ; [ 1.9565936   1.1691937   0.33301967 ...  0.24661978  1.3313323
 -0.8962238 ]


In [ ]:
print(textpkl['embeddings'][2],";",videopkl['embeddings'][2])

[-0.48289317 -0.3653778   0.19714946 ...  0.03739621 -0.674259
 -0.40728423] ; [-0.41057038  2.8972929   1.9146283  ... -0.3906505   1.564208
 -4.8234253 ]


In [ ]:
print(textpkl['embeddings'][3],";",videopkl['embeddings'][3])

[-0.9525771   0.19983402  0.39927948 ...  0.16337478 -0.04502395
 -0.29539204] ; [-1.1861308   0.53418887  0.04683519 ...  0.9691809   2.193101
 -0.7670548 ]


In [ ]:
!pip install transformers torch

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
!pip show clip

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    with open(os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename), 'r') as f:
        transcript = f.read()

    # Prepare the inputs
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)
    text_input = preprocess(transcript).unsqueeze(0).to(device)

    # Calculate features
    with torch.no_grad():
        video_features = model.encode_image(video_input)
        text_features = model.encode_text(text_input)

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.T).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")

TypeError: ignored

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    with open(os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename), 'r') as f:
        transcript = f.read()

    # Prepare the inputs
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)
    text_input = preprocess(transcript).to(device)

    # Calculate features
    with torch.no_grad():
        video_features = model.encode_image(video_input)
        text_features = model.encode_text(text_input)

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.T).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    transcript_path = os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename)
    with open(transcript_path, 'r') as f:
        transcript = f.read()

    # Prepare the inputs
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)
    text_input = clip.tokenize(transcript).to(device)

    # Calculate features
    with torch.no_grad():
        video_features = model.encode_image(video_input)
        text_features = model.encode_text(text_input)

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.T).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")

RuntimeError: ignored

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    transcript_path = os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename)
    with open(transcript_path, 'r') as f:
        transcript = f.read()

    # Split the transcript into chunks
    max_tokens = model.context_length - 2  # Reserve 2 tokens for special tokens
    transcript_chunks = [transcript[i:i + max_tokens] for i in range(0, len(transcript), max_tokens)]

    # Calculate features for each transcript chunk
    text_features = None
    for chunk in transcript_chunks:
        # Prepare the inputs
        text_input = clip.tokenize(["a photo of a " + chunk]).to(device)

        # Calculate features
        with torch.no_grad():
            chunk_features = model.encode_text(text_input)

        # Concatenate the features
        if text_features is None:
            text_features = chunk_features
        else:
            text_features = torch.cat((text_features, chunk_features), dim=1)

    # Prepare the video input
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)

    # Calculate features for the video
    with torch.no_grad():
        video_features = model.encode_image(video_input)

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.T).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    transcript_path = os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename)
    with open(transcript_path, 'r') as f:
        transcript = f.read()

    # Split the transcript into chunks
    max_tokens = model.context_length - 2  # Reserve 2 tokens for special tokens
    transcript_chunks = [transcript[i:i + max_tokens] for i in range(0, len(transcript), max_tokens)]

    # Calculate features for each transcript chunk
    text_features = None
    for chunk in transcript_chunks:
        # Prepare the inputs
        text_input = clip.tokenize(["a photo of a " + chunk]).to(device)

        # Calculate features
        with torch.no_grad():
            chunk_features = model.encode_text(text_input)

        # Concatenate the features
        if text_features is None:
            text_features = chunk_features
        else:
            text_features = torch.cat((text_features, chunk_features), dim=1)

    # Prepare the video input
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)

    # Calculate features for the video
    with torch.no_grad():
        video_features = model.encode_image(video_input)

    # Reshape the text features
    text_features = text_features.unsqueeze(0)

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.transpose(-1, -2)).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")

In [ ]:
import os
import pickle
import torch
import clip

# Load the video and text embeddings from pickle files
with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/videopad18.pickle', 'rb') as f:
    video_data = pickle.load(f)

with open('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/textpad18.pkl', 'rb') as f:
    text_data = pickle.load(f)

# Load the CLIP model and preprocessing function
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Prepare lists to store the video captions
video_captions = []

# Process video and text embeddings
for video_item in video_data:
    video_embedding = video_item['embeddings']
    video_filename = video_item['filename']
    video_suffix = video_filename[6:-4]  # Extract the suffix from video filename
    text_filename = f"assets-{video_suffix}.txt"

    # Load the transcript file for text data
    transcript_path = os.path.join('/content/drive/MyDrive/Tvnv1/CLIP (video-text)/text18test', text_filename)
    with open(transcript_path, 'r') as f:
        transcript = f.read()

    # Split the transcript into chunks
    max_tokens = model.context_length - 2  # Reserve 2 tokens for special tokens
    transcript_chunks = [transcript[i:i + max_tokens] for i in range(0, len(transcript), max_tokens)]

    # Calculate features for each transcript chunk
    text_features = None
    for chunk in transcript_chunks:
        # Prepare the inputs
        text_input = clip.tokenize(["a photo of a " + chunk]).to(device)

        # Calculate features
        with torch.no_grad():
            chunk_features = model.encode_text(text_input)

        # Concatenate the features
        if text_features is None:
            text_features = chunk_features
        else:
            text_features = torch.cat((text_features, chunk_features), dim=1)

    # Prepare the video input
    video_input = torch.from_numpy(video_embedding).unsqueeze(0).to(device)

    # Calculate features for the video
    with torch.no_grad():
        video_features = model.encode_image(video_input)

    # Reshape the text features
    text_features = text_features.view(1, -1, text_features.shape[-1])

    # Compute similarity scores between video and text features
    video_features /= video_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * video_features @ text_features.transpose(-1, -2)).softmax(dim=-1)

    # Select the most relevant captions
    values, indices = similarity[0].topk(5)
    captions = []
    for value, index in zip(values, indices):
        caption = video_filename  # Replace with appropriate field for captions
        captions.append(caption)

    # Generate video paragraph caption
    video_captions.append(' '.join(captions))

# Print the video paragraph captions
for i, caption in enumerate(video_captions):
    print(f"Video {i+1}: {caption}")